In [1]:
# Imports
import pandas as pd
import numpy as np
import re
import calendar
from collections import Counter
from collections import OrderedDict

# Opening cleaned dataset
df=pd.read_csv('Cleaned dataset.csv')

# Obtaining city from Location
city_dict={'Mumbai Metropolitan Region' : 'Mumbai',
            'Pune/Pimpri-Chinchwad Area' : 'Pune',
            'India' : 'Not mentioned',
            'Chandigarh' : 'Chandigarh',
            'Punjab' : 'Not mentioned',
            'Karnataka' : 'Not mentioned',
            'Dadra and Nagar Haveli' : 'Dadra and Nagar Haveli',
            'Goa' : 'Goa'}

def city(x):
    c=x.split(',')
    c1=c[0]
    if len(c)>2:
        return c1
    elif c1 in city_dict.keys():
        return city_dict[c1]
    elif 'Greater' in c1:
        return c1.split(' ')[1]
    
df['City']=df['Location'].apply(city)


# Obtaining state from location
state_dict={'Mumbai Metropolitan Region':'Maharashtra',
            'Greater Nashik Area':'Maharashtra',
            'Pune/Pimpri-Chinchwad Area':'Maharashtra',
            'Greater Madurai Area':'Tamil Nadu',
            'Greater Delhi Area':'Delhi',
            'Greater Hyderabad Area':'Telangana',
            'India':'Not mentioned'}

def state(x):
    s=x.split(',')
    if len(s)>2:
        return s[1].replace(' ','',1)
    elif len(s)==2:
        return s[0]
    elif x in state_dict.keys():
        return (state_dict[x])
df['State']=df['Location'].apply(state)


# Grouped city from city
def grp(x):
    if 'Bangalore Urban' in x or 'Bengaluru North' in x or 'Bengaluru Rural' in x:
        return 'Bengaluru'
    else:
        return x

df['Grouped City']=df['City'].apply(grp)


# Obtaining days from time posted
def days(x):
    if x!='Not available':
        if 'w' in x:
            return int(x.split(' ')[0])*7
        elif 'm' in x:
            return int(x.split(' ')[0])*31
        elif 'd' in x:
            return int(x.split(' ')[0])
    else:
        return 0
df['No. of days ago']=df['Time posted'].apply(days)

# Obtaining month posted from days
def month(x):
    if x!=0:
        td=pd.Timedelta(days=x)
        m=ts-td
        return (calendar.month_name[m.month])
    else:
        return 'Not mentioned'
    
ts=pd.Timestamp('2021/11/29') # Date on which data was scrapped from linkedin
df['Month posted']=df['No. of days ago'].apply(month)

# Counting number of time a individual term mentioned in job function
val=df['Job function']
terms=[]
for i in val:
    l=re.split(',| and',i)
    for ii in range(len(l)):
        if ii>0:
            terms.append(l[ii].replace(' ','',1))
        else:
            terms.append(l[ii])

terms_counter=Counter(terms)
terms_counter=dict(terms_counter)
del terms_counter['']
del terms_counter['Not mentioned']

# Creating a dataframe for obtained result for export
job_functions=pd.DataFrame({'Functions':terms_counter.keys(),'Total mentions':terms_counter.values()})

# Counting number of time a individual term mentioned in Industry
val=df['Industry']
terms=[]
for i in val:
    ll=i.replace(', and',',')
    l=re.split(',',ll)
    for ii in range(len(l)):
        if ii>0:
            terms.append(l[ii].replace(' ','',1))
        else:
            terms.append(l[ii])

terms_counter=Counter(terms)
terms_dict=dict(terms_counter)
del terms_dict['Not mentioned']
terms_dict

# Creating a dataframe for obtained result for export
industry=pd.DataFrame({'Industry':terms_dict.keys(),'Total mentions':terms_dict.values()})


# Modifying columns
del df['Location']
del df['Time posted']
    # Rearranging columns
column=['Role', 'Company', 'City','Grouped City','State','Month posted','No. of days ago', 'Level', 'Job type','Job function', 'Industry']
df=df[column]

In [78]:
# Exporting datasets
df.to_csv('Modified dataset.csv',index=False)
job_functions.to_csv('Job function count.csv', index=False)
industry.to_csv('Industry count.csv', index=False)